In [4]:
import gradio as gr
import joblib
import numpy as np
import pandas as pd
import os, sys
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import parselmouth
from parselmouth.praat import call

In [5]:
model = joblib.load( r'C:\Users\ASUS\Downloads\Detection of parkinsons disease using voice\xgboostmodel.pkl')

In [6]:


def greet(speech):
    array = np.asarray(speech)
    array2 = array[1]
    array3 = array2.astype(dtype = np.float64)
    sound2 = parselmouth.Sound(array3, 44100.0, 0.0) # read the sound
    def measurePitch(voiceID, f0min, f0max, unit):
        sound = voiceID 
        pitch = call(sound, "To Pitch", 0.0, f0min, f0max) #create a praat pitch object
        harmonicity = call(sound, "To Harmonicity (cc)", 0.01, 75, 0.1, 1.0)
        pointProcess = call(sound, "To PointProcess (periodic, cc)", f0min, f0max)
        localJitter = (call(pointProcess, "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3)) * 100
        localabsoluteJitter = call(pointProcess, "Get jitter (local, absolute)", 0, 0, 0.0001, 0.02, 1.3)
        rapJitter = (call(pointProcess, "Get jitter (rap)", 0, 0, 0.0001, 0.02, 1.3)) * 100
        ppq5Jitter = (call(pointProcess, "Get jitter (ppq5)", 0, 0, 0.0001, 0.02, 1.3)) * 100
        ddpJitter = (call(pointProcess, "Get jitter (ddp)", 0, 0, 0.0001, 0.02, 1.3)) * 100
        localShimmer =  (call([sound, pointProcess], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6)) * 100
        localdbShimmer = call([sound, pointProcess], "Get shimmer (local_dB)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
        apq3Shimmer = (call([sound, pointProcess], "Get shimmer (apq3)", 0, 0, 0.0001, 0.02, 1.3, 1.6)) * 100
        aqpq5Shimmer = (call([sound, pointProcess], "Get shimmer (apq5)", 0, 0, 0.0001, 0.02, 1.3, 1.6)) * 100
        apq11Shimmer =  (call([sound, pointProcess], "Get shimmer (apq11)", 0, 0, 0.0001, 0.02, 1.3, 1.6)) * 100
        ddaShimmer = (call([sound, pointProcess], "Get shimmer (dda)", 0, 0, 0.0001, 0.02, 1.3, 1.6)) * 100

        meanF0 = call(pitch, "Get mean", 0, 0, unit) # get mean pitch
        stdevF0 = call(pitch, "Get standard deviation", 0 ,0, unit) # get standard deviation

        hnr = call(harmonicity, "Get mean", 0, 0)


        return localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, ddpJitter, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer, hnr,meanF0, stdevF0
    (localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, ddpJitter, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer, hnr,meanF0, stdevF0 ) = measurePitch(sound2, 75, 500, "Hertz")
    newinput=[[localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, ddpJitter, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer, hnr,meanF0, stdevF0]]
    #scalar1=MinMaxScaler((-1,1))
    #nn=scalar1.fit_transform(newinput)

    output=model.predict(newinput)
    if output == 1:
        answer = "You have Parkinsons"
    else:
        answer = "You don't have Parkinsons"
    return answer

with gr.Blocks(css=".gradio-container {background-image: url('file=C:Users/ASUS/Downloads/resume-2-master/resume-2-master/img/background_image.png')}") as demo:
    gr.Markdown(
    """
    # Voice Based Parkinson's Disease Detection Using XGBoost Algorithm
    
    """)
    iface = gr.Interface( fn=greet, inputs="audio",outputs="text")
    
    
demo.launch()
    




Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\new folder\lib\site-packages\starlette\responses.py", line 335, in __call__
    stat_result = await anyio.to_thread.run_sync(os.stat, self.path)
  File "c:\new folder\lib\site-packages\anyio\to_thread.py", line 28, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(func, *args, cancellable=cancellable,
  File "c:\new folder\lib\site-packages\anyio\_backends\_asyncio.py", line 818, in run_sync_in_worker_thread
    return await future
  File "c:\new folder\lib\site-packages\anyio\_backends\_asyncio.py", line 754, in run
    result = context.run(func, *args)
FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\ASUS\\Users\\ASUS\\Downloads\\resume-2-master\\resume-2-master\\img\\background_image.png'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\new folder\lib\site-packages\uvicorn\pr